In [ ]:
import pandas as pd
import numpy as np
import re
import seaborn as sns
pizza_sales=pd.read_csv("../data/raw/pizza_sales.csv")

pizza_sales.head(1)


In [ ]:
#Creamos una función para numerar a las pizzas y los tamaños
#Tamaño de las pizzas
#Vemos cuantos tamaños hay: 
pizza_sales["pizza_size"].unique()
#Hay 5 por lo que aplicaremos la siguiente función:
def pizza_size (size):
    if size == "S":
        return 1
    elif size == "M":
        return 2
    elif size == "L":
        return 3
    elif size == "XL":
        return 4
    else:
        return 5
    

In [ ]:
#Pruebo mi función
pizza_sales["size_id"]=pizza_sales["pizza_size"].apply(pizza_size)
pizza_sales.head()

In [ ]:
#Miro cuántos valores de nombre de pizza hay 
pizza_sales["pizza_name"].unique()

#Me creo un dataframe sólo con el nombre de las pizzas
pizza_menu=pd.DataFrame(pizza_sales["pizza_name"])
pizza_menu.drop_duplicates(inplace=True)
#Las número, creo pizza_name_id
pizza_menu["pizza_name_id"]=np.arange(1,len(pizza_menu)+1)
pizza_menu

#La añado a la tabla principal
pizza_sales=pd.merge(pizza_sales,pizza_menu, on ="pizza_name", how ="left")
pizza_sales.head()

    

In [ ]:
# Creo una columna para el id de la pizza que es una combinación entre pizza_name_id y size_id
pizza_sales["id_final_pizza"]=pizza_sales["pizza_name_id_y"] + pizza_sales["size_id"]/10

# Cast IDs and quantity as integers
pizza_sales["pizza_id"] = pizza_sales["pizza_id"].apply(lambda x: int(x))
pizza_sales["order_id"] = pizza_sales["order_id"].apply(lambda x: int(x))
pizza_sales["quantity"] = pizza_sales["quantity"].apply(lambda x: int(x))

# Cast date
pizza_sales["order_date"] = pd.to_datetime(pizza_sales["order_date"], format="mixed")

#Cast time
pizza_sales["order_time"] = pd.to_datetime(pizza_sales["order_time"], format='%H:%M:%S').dt.time

display(pizza_sales)

In [ ]:
#Creo el csv orders
#Creo la base de la tabla order y quito los duplicados
orders=pizza_sales[["order_id","order_date", "order_time",]]
orders=orders.drop_duplicates()
orders = orders.reset_index(drop=True)

#Creo un dataframe con el número de pedido y el sumatorio del total del order_id

sum_order=(pizza_sales[["order_id","total_price"]]).groupby("order_id", as_index=False)["total_price"].sum()


#Hago merge con la otra tabla base

orders= pd.merge(orders, sum_order, on ="order_id", how="left")
orders

#Guardo el fichero
orders.to_csv('../data/clean/orders.csv', index=False)

In [ ]:
#Creo el csv pizza_orders sería la refería a las sales total
pizza_orders=pizza_sales[["pizza_id","order_id", "id_final_pizza","quantity"]]
pizza_orders
pizza_orders.to_csv('../data/clean/pizza_orders.csv', index=False)

In [ ]:
#Creo el csv pizzas_menu que sólo tiene información del tipo de pizza, los ingredientes y la categoría
pizzas_menu=pizza_sales[["pizza_name_id_y","pizza_name","pizza_category","pizza_ingredients"]]
pizzas_menu.drop_duplicates(inplace=True)
pizzas_menu=pizzas_menu.reset_index(drop=True)
pizzas_menu
pizzas_menu.to_csv('../data/clean/pizzas_menu.csv', index=False)

In [ ]:
#Creamos el csv pizza_info
pizza_info=pizza_sales[["id_final_pizza","pizza_name","size_id","unit_price","pizza_name_id_y"]]
pizza_info.drop_duplicates(inplace=True)
pizza_info
pizza_info.to_csv('../data/clean/pizza_info.csv', index=False)

In [ ]:
#Creamos la tabla size
sizes=pizza_sales[["size_id","pizza_size"]]
sizes.drop_duplicates(inplace=True)
sizes
sizes.to_csv('../data/clean/sizes.csv', index=False)

In [ ]:
pizza_sales.dtypes